<a href="https://colab.research.google.com/github/Abhijith-Nagarajan/PadhAI_Implementations/blob/main/PyTorch_Pretrained.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Objective ###

*   Run pretrained models to fine tune last layers.
*   Experiment with VGG, Inception and ResNet




#### Using Pre-Trained models

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import torch
import torchvision
import torchvision.datasets as datasets
import torchvision.models as models
import torchvision.transforms as transforms

In [3]:
import torch.optim as optim
import torch.nn as nn

##### Building transforms for VGG and ResNet #####

In [4]:
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),
                         (0.5,0.5,0.5)),
])

In [5]:
test_transform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),
                         (0.5,0.5,0.5)),
])

##### Building the transforms for Inception #####

In [7]:
train_transform_inception = transforms.Compose([
    transforms.RandomResizedCrop(229),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),
                         (0.5,0.5,0.5)),
])

In [8]:
test_transform_inception = transforms.Compose([
    transforms.RandomResizedCrop(229),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),
                         (0.5,0.5,0.5)),
])

##### Loading the dataset and data loaders #####

In [6]:
train_df = datasets.CIFAR10(root='./data',train = True, download = True, transform = train_transform)
test_df = datasets.CIFAR10(root='./data', train = False, download = True, transform = test_transform)

100%|██████████| 170498071/170498071 [00:17<00:00, 9922420.48it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [9]:
train_loader = torch.utils.data.DataLoader(train_df, batch_size = 8, shuffle = True)

In [10]:
test_loader = torch.utils.data.DataLoader(test_df, batch_size = 16, shuffle = True)

In [22]:
for i, data in enumerate(train_loader,0):
    image, label = data

    print(label)
    if i==1:
        break

tensor([2, 2, 3, 5, 3, 3, 3, 3])
tensor([7, 3, 1, 6, 8, 9, 3, 5])


##### VGG Model #####

In [11]:
vgg_model = models.vgg16_bn()

In [12]:
print(vgg_model)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(128, 256

In [13]:
vgg_model.classifier[6]

Linear(in_features=4096, out_features=1000, bias=True)

In [14]:
vgg_model.classifier[6] = nn.Linear(in_features=4096, out_features = 10)

In [15]:
vgg_model.classifier[6]

Linear(in_features=4096, out_features=10, bias=True)

In [16]:
vgg_model_pretrained = models.vgg16_bn(pretrained = True)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_BN_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_BN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16_bn-6c64b313.pth" to /root/.cache/torch/hub/checkpoints/vgg16_bn-6c64b313.pth
100%|██████████| 528M/528M [00:25<00:00, 21.5MB/s]


In [17]:
for param in vgg_model_pretrained.parameters():
    param.requires_grad = False

In [18]:
vgg_model_pretrained.classifier[6] = nn.Linear(in_features = 4096,out_features = 10)

In [19]:
for param in vgg_model_pretrained.parameters():
    if param.requires_grad:
        print(param.shape)

torch.Size([10, 4096])
torch.Size([10])


In [20]:
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(vgg_model_pretrained.parameters(),lr=0.001)

In [ ]:
def training_loop(dataloader, model):
    for i, data in enumerate(dataloader,0):
        images, labels = data
        optimizer.zero_grad()

        output = vgg_model_pretrained(images)
        loss = loss_fn(labels,output)
        loss.backward()

        optimizer.step()
        predicted_class = torch.max()
